In [3]:
import json

In [7]:
list = []
for line in open('../data/Sports_and_Outdoors_Reviews_training.json', 'r'):
    list.append(json.loads(line))

In [8]:
list

[{'overall': 3.0,
  'verified': True,
  'reviewTime': '08 22, 2017',
  'reviewerID': '59C5FEF4BC97055C3738F2364F8F7944',
  'asin': '4B3F7F19EBBF9167126A59036575E9B4',
  'style': {'Color:': ' Golden'},
  'reviewerName': '215FA0CD5C9A1CEFAB90AEC5E391E259',
  'reviewText': "A little loose, but cheep. So I'm OK with that",
  'summary': 'Three Stars',
  'unixReviewTime': 1503360000},
 {'overall': 5.0,
  'verified': True,
  'reviewTime': '07 20, 2017',
  'reviewerID': 'F0C7171929DC1E6C33EB04AD0774F810',
  'asin': '4B3F7F19EBBF9167126A59036575E9B4',
  'style': {'Color:': ' Black'},
  'reviewerName': 'B92DE484E5AF5CD97FFF91917D7133B1',
  'reviewText': 'A great replacement for the junk plastic caps that came with my presta tubes.  The plastic caps stripped and broke, making them useless.',
  'summary': 'Much better than plastic caps.',
  'unixReviewTime': 1500508800},
 {'overall': 5.0,
  'verified': True,
  'reviewTime': '03 13, 2017',
  'reviewerID': '713D0B36E3CE39956F7C0097911E4DBF',
  'asin